<a href="https://colab.research.google.com/github/Aviral-Aggarwal/Retail_models/blob/master/Retail_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basic Functions and loading

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
#New version of numpy is there, using the upgraded one
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random
import matplotlib.pyplot as plt
%matplotlib inline

from itertools import cycle

from math import ceil

from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler

warnings.filterwarnings('ignore')

#the sales training data has more than 1900 columns
pd.set_option('max_columns', 100)
#These are used for better plotting, check them later or ask about them
plt.style.use('bmh')
color_pal = plt.rcParams['axes.prop_cycle'].by_key()['color']
color_cycle = cycle(plt.rcParams['axes.prop_cycle'].by_key()['color'])

In [ ]:
input_dir = '/content/gdrive/My Drive/m5-forecasting-accuracy/'

#CHECK AGAIN HOW THIS IS WORKING, IS IT ZIP MECHANISM OR SOMETHING ELSE
#Standard function, copied from stack exchange
def reduce_mem_usage(df, verbose=False):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics: 
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
def read_data(input_dir):
    sell_prices_df = pd.read_csv(input_dir+'sell_prices.csv')
    sell_prices_df = reduce_mem_usage(df=sell_prices_df, verbose=True)
    print('The shape of sell prices is ', sell_prices_df.shape)
    
    sales_train_validation_df = pd.read_csv(input_dir+'sales_train_validation.csv')
    sales_train_validation_df = reduce_mem_usage(df=sales_train_validation_df, verbose=True)
    print('The shape of sales data is ', sales_train_validation_df.shape)
    
    calendar_df = pd.read_csv(input_dir+'calendar.csv')
    calendar_df = reduce_mem_usage(df=calendar_df, verbose=True)
    print('The shape of the calendar is ', calendar_df.shape)
    
    return sell_prices_df, sales_train_validation_df, calendar_df

Use the next function directly to load the combined dataframe and work on that

In [ ]:
def load_df(file_path):
    t1 = time.time()
    df = pd.read_csv(file_path)
    df = reduce_mem_usage(df=df, verbose=True)
    print('The shape of food_master is ', df.shape)
    t2 = time.time()
    print('\nTime taken to load the DataFrame is:', (t2-t1))
    return df

In [ ]:
t1 = time.time()
sell_df, sales_df, calendar = read_data(input_dir)
t2 = time.time()
print('Time taken for loading data is ', (t2-t1))

Mem. usage decreased to 130.48 Mb (37.5% reduction)
The shape of sell prices is  (6841121, 4)
Mem. usage decreased to 95.00 Mb (78.7% reduction)
The shape of sales data is  (30490, 1919)
Mem. usage decreased to  0.12 Mb (41.9% reduction)
The shape of the calendar is  (1969, 14)
Time taken for loading data is  114.0935001373291


The Sales and prices dataframe is only till d_1913, so I am taking calendar also till that value only

In [ ]:
calendar_test = calendar[1914:]
calendar = calendar[0:1913]

In [ ]:
calendar

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1908,2016-04-20,11612,Wednesday,5,4,2016,d_1909,NaN,NaN,NaN,NaN,0,0,0
1909,2016-04-21,11612,Thursday,6,4,2016,d_1910,NaN,NaN,NaN,NaN,0,0,0
1910,2016-04-22,11612,Friday,7,4,2016,d_1911,NaN,NaN,NaN,NaN,0,0,0
1911,2016-04-23,11613,Saturday,1,4,2016,d_1912,NaN,NaN,NaN,NaN,0,0,0


In [ ]:
sales_df

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,d_40,d_41,d_42,d_43,d_44,...,d_1864,d_1865,d_1866,d_1867,d_1868,d_1869,d_1870,d_1871,d_1872,d_1873,d_1874,d_1875,d_1876,d_1877,d_1878,d_1879,d_1880,d_1881,d_1882,d_1883,d_1884,d_1885,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,1,3,0,0,0,1,1,1,3,1,3,1,2,2,0,1,1,1,1,0,0,0,0,0,1,0,4,2,3,0,1,2,0,0,0,1,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,2,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,2,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,2,0,1,0,...,5,3,1,0,0,0,1,2,3,0,1,3,4,2,1,4,1,3,5,0,6,6,0,0,0,0,3,1,2,1,3,1,0,2,5,4,2,0,3,0,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,2,1,1,0,3,1,1,2,1,1,0,3,2,2,2,3,1,0,0,0,0,1,0,4,4,0,1,4,0,1,0,1,0,1,1,2,0,1,1,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,0,3,1,4,1,0,0,3,4,4,0,0,1,0,1,1,7,7,3,6,3,3,7,12,4,2,7,5,12,5,3,3,7,6,4,6,3,6,2,3,...,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,2,0,0,0,0,0,1,0,0,1
30486,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,0,5,0,1,1,3,1,1,0,4,2,0,1,2,1,1,0,0,0,0,3,1,1,1,2,0,1,0,1,1,1,3,3,4,4,3,6,1,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,2,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0
30487,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,2,4,1,8,5,2,7,5,3,5,20,8,10,3,3,4,7,2,3,5,6,3,4,1,2,5,1,2,2,2,1,9,8,1,6,3,3,4,10,4,...,2,1,1,2,1,3,1,1,0,3,1,1,2,0,0,2,0,0,0,2,0,1,1,0,0,0,0,1,1,2,1,4,0,0,2,2,4,0,0,0,2,1,0,2,0,1,0,0,1,0
30488,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,1,3,0,3,1,1,1,1,1,1,0,2,1,1,2,4,3,0,0,0,0,0,1,1,2,1,1,1,1,2,0,1,0,3,0,0,1,0,0,1,0,3,1,3


In [ ]:
list_dept = sales_df['dept_id'].unique()

In [ ]:
list_dept

array(['HOBBIES_1', 'HOBBIES_2', 'HOUSEHOLD_1', 'HOUSEHOLD_2', 'FOODS_1',
       'FOODS_2', 'FOODS_3'], dtype=object)

# Merging and sorting

In [ ]:
food_sales_df = sales_df[sales_df['dept_id']=='HOBBIES_2']
#Without the next statement the index column of the new DF goes from 1612 to 30489, creates issues later
food_sales_df.reset_index(inplace = True)
food_sales_df.drop('cat_id', axis=1, inplace=True)
food_sales_df.drop('index', axis=1, inplace=True)

In [ ]:
food_sales_df

,id,item_id,dept_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,d_40,d_41,d_42,d_43,d_44,d_45,...,d_1864,d_1865,d_1866,d_1867,d_1868,d_1869,d_1870,d_1871,d_1872,d_1873,d_1874,d_1875,d_1876,d_1877,d_1878,d_1879,d_1880,d_1881,d_1882,d_1883,d_1884,d_1885,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_2_001_CA_1_validation,HOBBIES_2_001,HOBBIES_2,CA_1,CA,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,1,0
1,HOBBIES_2_002_CA_1_validation,HOBBIES_2_002,HOBBIES_2,CA_1,CA,4,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,3,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,2,1,...,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_2_003_CA_1_validation,HOBBIES_2_003,HOBBIES_2,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2,0
3,HOBBIES_2_004_CA_1_validation,HOBBIES_2_004,HOBBIES_2,CA_1,CA,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1,0,1,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,1,1,0,1,0,1,0,0,0,0,0,1,1
4,HOBBIES_2_005_CA_1_validation,HOBBIES_2_005,HOBBIES_2,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,1,1,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1485,HOBBIES_2_145_WI_3_validation,HOBBIES_2_145,HOBBIES_2,WI_3,WI,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,1,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,1,0,0,4,1,1,0,0,0,0,0,0,0,0,0,0,2,0,1,0,1,0,1,0,1,2,0,1,0,0,0,0,2,0,3,0,0,0,0,0,0,0,0,0,4,0,0,0
1486,HOBBIES_2_146_WI_3_validation,HOBBIES_2_146,HOBBIES_2,WI_3,WI,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,1,0,1,...,0,0,0,1,0,0,1,0,0,0,0,0,1,0,2,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1487,HOBBIES_2_147_WI_3_validation,HOBBIES_2_147,HOBBIES_2,WI_3,WI,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,3,2,0,0,2,2,0,1,0,1,0,0,0,1,1,0,0,1,0,3,0,0,0,1,1,0,0,0,4,3,0,0,0,1,3,1,0,0,0,0,2,0,1,2,0
1488,HOBBIES_2_148_WI_3_validation,HOBBIES_2_148,HOBBIES_2,WI_3,WI,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2,1,0,0,0,2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,4,0,1,0,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
food_prc_df = sell_df[sell_df.item_id.str.contains('HOBBIES_2', case=False)]
food_prc_df.reset_index(inplace=True)
food_prc_df.drop('index', axis=1, inplace=True)

In [ ]:
food_prc_df

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_2_001,11101,5.468750
1,CA_1,HOBBIES_2_001,11102,5.468750
2,CA_1,HOBBIES_2_001,11103,5.468750
3,CA_1,HOBBIES_2_001,11104,5.468750
4,CA_1,HOBBIES_2_001,11105,5.468750
...,...,...,...,...
323563,WI_3,HOBBIES_2_149,11617,0.970215
323564,WI_3,HOBBIES_2_149,11618,0.970215
323565,WI_3,HOBBIES_2_149,11619,0.970215
323566,WI_3,HOBBIES_2_149,11620,0.970215


In [ ]:
#food_prc_test = food_prc_df[(food_prc_df['wm_yr_wk']>=11614)]
food_prc_df = food_prc_df[(food_prc_df['wm_yr_wk']<=11613)]

In [ ]:
food_prc_df

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_2_001,11101,5.468750
1,CA_1,HOBBIES_2_001,11102,5.468750
2,CA_1,HOBBIES_2_001,11103,5.468750
3,CA_1,HOBBIES_2_001,11104,5.468750
4,CA_1,HOBBIES_2_001,11105,5.468750
...,...,...,...,...
323555,WI_3,HOBBIES_2_149,11609,0.970215
323556,WI_3,HOBBIES_2_149,11610,0.970215
323557,WI_3,HOBBIES_2_149,11611,0.970215
323558,WI_3,HOBBIES_2_149,11612,0.970215


In [ ]:
food_sales_df.groupby(by=['dept_id', 'item_id', 'store_id']).sum()

d_1  d_2  d_3  d_4  d_5  d_6  d_7  d_8  d_9  \
dept_id   item_id       store_id                                                
HOBBIES_2 HOBBIES_2_001 CA_1        1    0    0    0    0    1    0    0    0   
                        CA_2        0    0    0    0    0    0    0    0    0   
                        CA_3        0    0    0    0    0    0    0    0    0   
                        CA_4        0    0    0    0    0    0    0    0    0   
                        TX_1        0    0    0    0    0    0    0    0    0   
...                               ...  ...  ...  ...  ...  ...  ...  ...  ...   
          HOBBIES_2_149 TX_2        0    0    0    0    0    0    0    0    0   
                        TX_3        0    0    0    0    0    0    0    0    0   
                        WI_1        0    0    0    0    0    0    0    0    0   
                        WI_2        0    0    0    0    0    0    0    0    0   
                        WI_3        0    0    0    0    0    0    0    0    0   

                                  d_10  d_11  d_12  d_13  d_14  d_15  d_16  \
dept_id   item_id       store_id                                             
HOBBIES_2 HOBBIES_2_001 CA_1         0     0     0     0     0     0     0   
                        CA_2         0     0     0     0     0     0     0   
                        CA_3         0     0     0     0     0     0     0   
                        CA_4         0     0     0     0     0     0     0   
                        TX_1         0     0     0     0     1     0     0   
...                                ...   ...   ...   ...   ...   ...   ...   
          HOBBIES_2_149 TX_2         0     0     0     0     0     0     0   
                        TX_3         0     0     0     0     0     0     0   
                        WI_1         0     0     0     0     0     0     0   
                        WI_2         0     0     0     0     0     0     0   
                        WI_3         0     0     0     0     0     0     0   

                                  d_17  d_18  d_19  d_20  d_21  d_22  d_23  \
dept_id   item_id       store_id                                             
HOBBIES_2 HOBBIES_2_001 CA_1         0     0     0     0     0     0     0   
                        CA_2         0     0     0     0     0     0     0   
                        CA_3         0     0     0     0     0     0     0   
                        CA_4         1     0     0     0     0     0     0   
                        TX_1         0     0     0     0     0     0     0   
...                                ...   ...   ...   ...   ...   ...   ...   
          HOBBIES_2_149 TX_2         0     0     0     0     0     0     0   
                        TX_3         0     0     0     0     0     0     0   
                        WI_1         0     0     0     0     0     0     0   
                        WI_2         0     0     0     0     0     0     0   
                        WI_3         0     0     0     0     0     0     0   

                                  d_24  d_25  d_26  d_27  d_28  d_29  d_30  \
dept_id   item_id       store_id                                             
HOBBIES_2 HOBBIES_2_001 CA_1         2     0     0     0     1     1     0   
                        CA_2         0     0     0     0     0     0     0   
                        CA_3         0     0     0     0     0     0     0   
                        CA_4         0     0     0     0     0     0     1   
                        TX_1         0     0     0     0     0     0     0   
...                                ...   ...   ...   ...   ...   ...   ...   
          HOBBIES_2_149 TX_2         0     0     0     0     0     0     0   
                        TX_3         0     0     0     0     0     0     0   
                        WI_1         0     0     0     0     0     0     0   
                        WI_2         0     0     0     0     0     0     0   
              

# Creating the feature columns

Calendar has a column for the days (d_1, d_2, ..., d_1913) and this is in the form of rows in the food_sales_df. I have to combine both of them.



Features to be added as per the last call:


1.   Month
2.   Week number of year
3. Weekday
4. Price reduction or discount
5. Price elasticity of demand
6. Yearly government holidays
7. Columns for each of the holidays and then the time after that (-3 weeks and +2 weeks, these two being external parameters
8. 2D Features



In [ ]:
food_flat=food_sales_df.melt(['dept_id', 'item_id' ,'store_id', 'id', 'state_id'], var_name='d', value_name='Quantity_sold')

In [ ]:
food_flat.drop('state_id', axis=1, inplace = True)

In [ ]:
food_flat.sort_values(by = ['dept_id', 'item_id', 'store_id'], ascending= True, inplace = True)

In [ ]:
food_flat.reset_index(inplace = True)
food_flat

,index,dept_id,item_id,store_id,id,d,Quantity_sold
0,0,HOBBIES_2,HOBBIES_2_001,CA_1,HOBBIES_2_001_CA_1_validation,d_1,1
1,1490,HOBBIES_2,HOBBIES_2_001,CA_1,HOBBIES_2_001_CA_1_validation,d_2,0
2,2980,HOBBIES_2,HOBBIES_2_001,CA_1,HOBBIES_2_001_CA_1_validation,d_3,0
3,4470,HOBBIES_2,HOBBIES_2_001,CA_1,HOBBIES_2_001_CA_1_validation,d_4,0
4,5960,HOBBIES_2,HOBBIES_2_001,CA_1,HOBBIES_2_001_CA_1_validation,d_5,0
...,...,...,...,...,...,...,...
2850365,2844409,HOBBIES_2,HOBBIES_2_149,WI_3,HOBBIES_2_149_WI_3_validation,d_1909,2
2850366,2845899,HOBBIES_2,HOBBIES_2_149,WI_3,HOBBIES_2_149_WI_3_validation,d_1910,0
2850367,2847389,HOBBIES_2,HOBBIES_2_149,WI_3,HOBBIES_2_149_WI_3_validation,d_1911,0
2850368,2848879,HOBBIES_2,HOBBIES_2_149,WI_3,HOBBIES_2_149_WI_3_validation,d_1912,0


In [ ]:
date=calendar[['wm_yr_wk','d']].drop_duplicates()

In [ ]:
food_flat.drop('index', axis = 1, inplace=True)

In [ ]:
food_prc = pd.merge(food_prc_df, date, how='left', on=['wm_yr_wk'])

In [ ]:
food_prc

,store_id,item_id,wm_yr_wk,sell_price,d
0,CA_1,HOBBIES_2_001,11101,5.468750,d_1
1,CA_1,HOBBIES_2_001,11101,5.468750,d_2
2,CA_1,HOBBIES_2_001,11101,5.468750,d_3
3,CA_1,HOBBIES_2_001,11101,5.468750,d_4
4,CA_1,HOBBIES_2_001,11101,5.468750,d_5
...,...,...,...,...,...
2174081,WI_3,HOBBIES_2_149,11612,0.970215,d_1909
2174082,WI_3,HOBBIES_2_149,11612,0.970215,d_1910
2174083,WI_3,HOBBIES_2_149,11612,0.970215,d_1911
2174084,WI_3,HOBBIES_2_149,11613,0.970215,d_1912


The order is now first all the days of a single item at one store, then at next store; for all the years. After that at the next item at the first store and so on. I had a different order first where all the data was available for a single day in the order store_id-> item_id-> for each day and then the next day

In [ ]:
food_prc.shape

(2174086, 5)

In [ ]:
food_prc_df.shape

(311648, 4)

This now checks out. I think this relatively smaller discrepancy is because the last week has data only for 2 days and the remaining 5 are missing. And similar other cases.

In [ ]:
food_master = pd.merge(food_flat, food_prc, on=['d' ,'store_id' ,'item_id'],how='inner')

In [ ]:
calendar.drop('wm_yr_wk', axis=1, inplace=True)

In [ ]:
food_master = pd.merge(food_master, calendar, on=['d'],how='left')

In [ ]:
food_master.drop(['event_name_1', 'event_name_2', 'event_type_1', 'event_type_2'], axis=1, inplace = True)
food_master.drop(['snap_CA', 'snap_TX', 'snap_WI'], axis=1, inplace = True)

In [ ]:
food_master

,dept_id,item_id,store_id,id,d,Quantity_sold,wm_yr_wk,sell_price,date,weekday,wday,month,year
0,HOBBIES_2,HOBBIES_2_001,CA_1,HOBBIES_2_001_CA_1_validation,d_1,1,11101,5.468750,2011-01-29,Saturday,1,1,2011
1,HOBBIES_2,HOBBIES_2_001,CA_1,HOBBIES_2_001_CA_1_validation,d_2,0,11101,5.468750,2011-01-30,Sunday,2,1,2011
2,HOBBIES_2,HOBBIES_2_001,CA_1,HOBBIES_2_001_CA_1_validation,d_3,0,11101,5.468750,2011-01-31,Monday,3,1,2011
3,HOBBIES_2,HOBBIES_2_001,CA_1,HOBBIES_2_001_CA_1_validation,d_4,0,11101,5.468750,2011-02-01,Tuesday,4,2,2011
4,HOBBIES_2,HOBBIES_2_001,CA_1,HOBBIES_2_001_CA_1_validation,d_5,0,11101,5.468750,2011-02-02,Wednesday,5,2,2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2174081,HOBBIES_2,HOBBIES_2_149,WI_3,HOBBIES_2_149_WI_3_validation,d_1909,2,11612,0.970215,2016-04-20,Wednesday,5,4,2016
2174082,HOBBIES_2,HOBBIES_2_149,WI_3,HOBBIES_2_149_WI_3_validation,d_1910,0,11612,0.970215,2016-04-21,Thursday,6,4,2016
2174083,HOBBIES_2,HOBBIES_2_149,WI_3,HOBBIES_2_149_WI_3_validation,d_1911,0,11612,0.970215,2016-04-22,Friday,7,4,2016
2174084,HOBBIES_2,HOBBIES_2_149,WI_3,HOBBIES_2_149_WI_3_validation,d_1912,0,11613,0.970215,2016-04-23,Saturday,1,4,2016


In [ ]:
food_master['mrp'] = pd.DataFrame(food_master.groupby(['item_id', 'store_id'])['sell_price'].apply(lambda x: x.cummax()))

In [ ]:
food_master['discount'] = food_master['mrp'] - food_master['sell_price']

In [ ]:
food_master['date'] = pd.to_datetime(food_master['date'], errors='coerce')
food_master['week_number'] = food_master['date'].dt.week

In [ ]:
food_master

,dept_id,item_id,store_id,id,d,Quantity_sold,wm_yr_wk,sell_price,date,weekday,wday,month,year,mrp,discount,week_number
0,HOBBIES_2,HOBBIES_2_001,CA_1,HOBBIES_2_001_CA_1_validation,d_1,1,11101,5.468750,2011-01-29,Saturday,1,1,2011,5.468750,0.0,4
1,HOBBIES_2,HOBBIES_2_001,CA_1,HOBBIES_2_001_CA_1_validation,d_2,0,11101,5.468750,2011-01-30,Sunday,2,1,2011,5.468750,0.0,4
2,HOBBIES_2,HOBBIES_2_001,CA_1,HOBBIES_2_001_CA_1_validation,d_3,0,11101,5.468750,2011-01-31,Monday,3,1,2011,5.468750,0.0,5
3,HOBBIES_2,HOBBIES_2_001,CA_1,HOBBIES_2_001_CA_1_validation,d_4,0,11101,5.468750,2011-02-01,Tuesday,4,2,2011,5.468750,0.0,5
4,HOBBIES_2,HOBBIES_2_001,CA_1,HOBBIES_2_001_CA_1_validation,d_5,0,11101,5.468750,2011-02-02,Wednesday,5,2,2011,5.468750,0.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2174081,HOBBIES_2,HOBBIES_2_149,WI_3,HOBBIES_2_149_WI_3_validation,d_1909,2,11612,0.970215,2016-04-20,Wednesday,5,4,2016,2.470703,1.5,16
2174082,HOBBIES_2,HOBBIES_2_149,WI_3,HOBBIES_2_149_WI_3_validation,d_1910,0,11612,0.970215,2016-04-21,Thursday,6,4,2016,2.470703,1.5,16
2174083,HOBBIES_2,HOBBIES_2_149,WI_3,HOBBIES_2_149_WI_3_validation,d_1911,0,11612,0.970215,2016-04-22,Friday,7,4,2016,2.470703,1.5,16
2174084,HOBBIES_2,HOBBIES_2_149,WI_3,HOBBIES_2_149_WI_3_validation,d_1912,0,11613,0.970215,2016-04-23,Saturday,1,4,2016,2.470703,1.5,16


After creating the basic features, save it once and then from now on just use the master data frame instead of running the above cells

In [ ]:
food_master.to_csv(input_dir + 'food1.csv')

KeyboardInterrupt: ignored

# Events and other monthly columns

In [ ]:
calendar = load_df(input_dir + 'calendar.csv')

Mem. usage decreased to  0.12 Mb (41.9% reduction)
The shape of food_master is  (1969, 14)

Time taken to load the DataFrame is: 0.2546241283416748


In [ ]:
calendar = calendar[:1913]
calendar

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1908,2016-04-20,11612,Wednesday,5,4,2016,d_1909,NaN,NaN,NaN,NaN,0,0,0
1909,2016-04-21,11612,Thursday,6,4,2016,d_1910,NaN,NaN,NaN,NaN,0,0,0
1910,2016-04-22,11612,Friday,7,4,2016,d_1911,NaN,NaN,NaN,NaN,0,0,0
1911,2016-04-23,11613,Saturday,1,4,2016,d_1912,NaN,NaN,NaN,NaN,0,0,0


event_name_2 and event_type_2 have a subset of elements from the 1-indexed columns. I have no idea why there are two separate columns for these. I am only taking the superset because that is sufficient

In [ ]:
list_events = list(calendar['event_name_1'].unique())
list_events = [x for x in list_events if str(x) != 'nan']

list_event_type = list(calendar['event_type_1'].unique())
list_event_type = [x for x in list_event_type if str(x) != 'nan']

In [ ]:
events_no = len(list_events)

In [ ]:
list_event_type

['Sporting', 'Cultural', 'National', 'Religious']

In [ ]:
food = load_df()

TypeError: ignored

In [ ]:
food_list = list(food_sales_df['item_id'].unique())

In [ ]:
#This is the total number of food items present in all the dataframes
len(food_list)

216

In [ ]:
food_master.shape

(3530738, 16)

In [ ]:
food_master[(food_master['discount']!=0)]

,dept_id,item_id,store_id,id,d,Quantity_sold,wm_yr_wk,sell_price,date,PresidentsDay,LentStart,LentWeek2,StPatricksDay,Purim End,OrthodoxEaster,Pesach End,Cinco De Mayo,Mother's day,MemorialDay,NBAFinalsStart,NBAFinalsEnd,Father's day,IndependenceDay,Ramadan starts,Eid al-Fitr,LaborDay,ColumbusDay,Halloween,EidAlAdha,VeteransDay,Thanksgiving,Christmas,Chanukah End,NewYear,OrthodoxChristmas,MartinLutherKingDay,Easter,mrp,discount,week_number
35,HOUSEHOLD_1,HOUSEHOLD_1_001,CA_1,HOUSEHOLD_1_001_CA_1_validation,d_36,0,11106,6.269531,2011-03-05,12.0,-4.0,-11.0,-12.0,-15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.320312,0.050781,9
36,HOUSEHOLD_1,HOUSEHOLD_1_001,CA_1,HOUSEHOLD_1_001_CA_1_validation,d_37,0,11106,6.269531,2011-03-06,13.0,-3.0,-10.0,-11.0,-14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.320312,0.050781,9
37,HOUSEHOLD_1,HOUSEHOLD_1_001,CA_1,HOUSEHOLD_1_001_CA_1_validation,d_38,0,11106,6.269531,2011-03-07,NaN,-2.0,-9.0,-10.0,-13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.320312,0.050781,10
38,HOUSEHOLD_1,HOUSEHOLD_1_001,CA_1,HOUSEHOLD_1_001_CA_1_validation,d_39,0,11106,6.269531,2011-03-08,NaN,-1.0,-8.0,-9.0,-12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.320312,0.050781,10
39,HOUSEHOLD_1,HOUSEHOLD_1_001,CA_1,HOUSEHOLD_1_001_CA_1_validation,d_40,2,11106,6.269531,2011-03-09,NaN,0.0,-7.0,-8.0,-11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.320312,0.050781,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7797554,HOUSEHOLD_1,HOUSEHOLD_1_541,WI_3,HOUSEHOLD_1_541_WI_3_validation,d_1909,0,11612,6.968750,2016-04-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.968750,1.000000,16
7797555,HOUSEHOLD_1,HOUSEHOLD_1_541,WI_3,HOUSEHOLD_1_541_WI_3_validation,d_1910,0,11612,6.968750,2016-04-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.968750,1.000000,16
7797556,HOUSEHOLD_1,HOUSEHOLD_1_541,WI_3,HOUSEHOLD_1_541_WI_3_validation,d_1911,0,11612,6.968750,2016-04-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.968750,1.000000,16
7797557,HOUSEHOLD_1,HOUSEHOLD_1_541,WI_3,HOUSEHOLD_1_541_WI_3_validation,d_1912,0,11613,6.968750,2016-04-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.968750,1.000000,16


About 30% of the rows have discounts. This was more of a sanity check rather than calculation because I was not sure if it is calculating the discount correctly in the first place

In [ ]:
calendar['SuperBowl'] = pd.DataFrame(calendar['event_name_1']=='SuperBowl', dtype='int')

In [ ]:
calendar[(calendar['SuperBowl']==1)]

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,PresidentsDay,LentStart,LentWeek2,StPatricksDay,Purim End,OrthodoxEaster,Pesach End,Cinco De Mayo,Mother's day,MemorialDay,NBAFinalsStart,NBAFinalsEnd,Father's day,IndependenceDay,Ramadan starts,Eid al-Fitr,LaborDay,ColumbusDay,Halloween,EidAlAdha,VeteransDay,Thanksgiving,Christmas,Chanukah End,NewYear,OrthodoxChristmas,MartinLutherKingDay,Easter,SuperBowl
8,2011-02-06,11102,Sunday,2,2,2011,d_9,SuperBowl,Sporting,NaN,NaN,-15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
372,2012-02-05,11202,Sunday,2,2,2012,d_373,SuperBowl,Sporting,NaN,NaN,-15.0,-17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
736,2013-02-03,11302,Sunday,2,2,2013,d_737,SuperBowl,Sporting,NaN,NaN,-15.0,-10.0,-17.0,NaN,-21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,NaN,1
1100,2014-02-02,11401,Sunday,2,2,2014,d_1101,SuperBowl,Sporting,NaN,NaN,-15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,NaN,1
1464,2015-02-01,11501,Sunday,2,2,2015,d_1465,SuperBowl,Sporting,NaN,NaN,-15.0,-17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,NaN,1
1835,2016-02-07,11602,Sunday,2,2,2016,d_1836,SuperBowl,Sporting,NaN,NaN,-8.0,-3.0,-10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [ ]:
index = calendar.index[calendar['event_name_1']=='SuperBowl'].to_list()

In [ ]:
import numpy as np

In [ ]:
index

[8, 372, 736, 1100, 1464, 1835]

In [ ]:
for l in index:
  for t in range ((l-21),(l+14)):
    calendar['SuperBowl'][t] = (t-l)

In [ ]:
calendar['SuperBowl'].replace(0, np.NaN, inplace = True)

In [ ]:
for i in index:
  calendar['SuperBowl'][i] = 0

In [ ]:
calendar[0:43]

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,PresidentsDay,LentStart,LentWeek2,StPatricksDay,Purim End,OrthodoxEaster,Pesach End,Cinco De Mayo,Mother's day,MemorialDay,NBAFinalsStart,NBAFinalsEnd,Father's day,IndependenceDay,Ramadan starts,Eid al-Fitr,LaborDay,ColumbusDay,Halloween,EidAlAdha,VeteransDay,Thanksgiving,Christmas,Chanukah End,NewYear,OrthodoxChristmas,MartinLutherKingDay,Easter,SuperBowl
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,-21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,-20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,-19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
5,2011-02-03,11101,Thursday,6,2,2011,d_6,NaN,NaN,NaN,NaN,-18.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
6,2011-02-04,11101,Friday,7,2,2011,d_7,NaN,NaN,NaN,NaN,-17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
7,2011-02-05,11102,Saturday,1,2,2011,d_8,NaN,NaN,NaN,NaN,-16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
8,2011-02-06,11102,Sunday,2,2,2011,d_9,SuperBowl,Sporting,NaN,NaN,-15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
9,2011-02-07,11102,Monday,3,2,2011,d_10,NaN,NaN,NaN,NaN,-14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [ ]:
def event_feature(lower_days, upper_days):
  for event in list_events:
    calendar[event] = pd.DataFrame(calendar['event_name_1']==event, dtype='int')
    index = calendar.index[calendar['event_name_1']==event].to_list()

    #Assign the values to the range of columns
    for i in index:
      for t in range ((i-lower_days),(i+upper_days)):
        calendar[event][t] = (t-i)
    #Make the rest of the columns NaN
    calendar[event].replace(0, np.NaN, inplace=True)
    for i in index:
      calendar[event][i] = 0

In [ ]:
event_feature(21, 14)

I have chosen this particular range since four of these events are occuring together at short intervals

In [ ]:
calendar[306:347]

,date,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,SuperBowl,ValentinesDay,PresidentsDay,LentStart,LentWeek2,StPatricksDay,Purim End,OrthodoxEaster,Pesach End,Cinco De Mayo,Mother's day,MemorialDay,NBAFinalsStart,NBAFinalsEnd,Father's day,IndependenceDay,Ramadan starts,Eid al-Fitr,LaborDay,ColumbusDay,Halloween,EidAlAdha,VeteransDay,Thanksgiving,Christmas,Chanukah End,NewYear,OrthodoxChristmas,MartinLutherKingDay,Easter
306,2011-12-01,Thursday,6,12,2011,d_307,NaN,NaN,NaN,NaN,1,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN
307,2011-12-02,Friday,7,12,2011,d_308,NaN,NaN,NaN,NaN,1,0,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN
308,2011-12-03,Saturday,1,12,2011,d_309,NaN,NaN,NaN,NaN,1,1,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN
309,2011-12-04,Sunday,2,12,2011,d_310,NaN,NaN,NaN,NaN,1,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,-21.0,NaN,NaN,NaN,NaN,NaN
310,2011-12-05,Monday,3,12,2011,d_311,NaN,NaN,NaN,NaN,1,1,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,-20.0,NaN,NaN,NaN,NaN,NaN
311,2011-12-06,Tuesday,4,12,2011,d_312,NaN,NaN,NaN,NaN,1,1,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,-19.0,NaN,NaN,NaN,NaN,NaN
312,2011-12-07,Wednesday,5,12,2011,d_313,NaN,NaN,NaN,NaN,1,1,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,-18.0,-21.0,NaN,NaN,NaN,NaN
313,2011-12-08,Thursday,6,12,2011,d_314,NaN,NaN,NaN,NaN,1,0,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-17.0,-20.0,NaN,NaN,NaN,NaN
314,2011-12-09,Friday,7,12,2011,d_315,NaN,NaN,NaN,NaN,1,1,1,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-16.0,-19.0,NaN,NaN,NaN,NaN
315,2011-12-10,Saturday,1,12,2011,d_316,NaN,NaN,NaN,NaN,1,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-15.0,-18.0,NaN,NaN,NaN,NaN


In [ ]:
calendar.to_csv(input_dir + 'calendar_event.csv')

In [ ]:
calendar = load_df(input_dir + 'calendar_event.csv')

Mem. usage decreased to  0.23 Mb (65.3% reduction)
The shape of food_master is  (1913, 45)

Time taken to load the DataFrame is: 0.056569576263427734


In [ ]:
calendar.drop(['snap_TX', 'snap_CA', 'snap_WI', 'SuperBowl', 'ValentinesDay', 'event_name_2', 'event_type_1', 'event_type_2'], axis=1, inplace=True)

In [ ]:
calendar.drop(['weekday','wday', 'month', 'year'], axis=1, inplace=True)

In [ ]:
calendar.drop(calendar.columns[0], axis=1, inplace=True)

In [ ]:
calendar

,date,wm_yr_wk,d,event_name_1,PresidentsDay,LentStart,LentWeek2,StPatricksDay,Purim End,OrthodoxEaster,Pesach End,Cinco De Mayo,Mother's day,MemorialDay,NBAFinalsStart,NBAFinalsEnd,Father's day,IndependenceDay,Ramadan starts,Eid al-Fitr,LaborDay,ColumbusDay,Halloween,EidAlAdha,VeteransDay,Thanksgiving,Christmas,Chanukah End,NewYear,OrthodoxChristmas,MartinLutherKingDay,Easter
0,2011-01-29,11101,d_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2011-01-30,11101,d_2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2011-01-31,11101,d_3,NaN,-21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2011-02-01,11101,d_4,NaN,-20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2011-02-02,11101,d_5,NaN,-19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1908,2016-04-20,11612,d_1909,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1909,2016-04-21,11612,d_1910,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1910,2016-04-22,11612,d_1911,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1911,2016-04-23,11613,d_1912,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#food_master = load_df((input_dir+'food3.csv'))
food_master

,dept_id,item_id,store_id,id,d,Quantity_sold,wm_yr_wk,sell_price,date,weekday,wday,month,year,mrp,discount,week_number
0,HOBBIES_2,HOBBIES_2_001,CA_1,HOBBIES_2_001_CA_1_validation,d_1,1,11101,5.468750,2011-01-29,Saturday,1,1,2011,5.468750,0.0,4
1,HOBBIES_2,HOBBIES_2_001,CA_1,HOBBIES_2_001_CA_1_validation,d_2,0,11101,5.468750,2011-01-30,Sunday,2,1,2011,5.468750,0.0,4
2,HOBBIES_2,HOBBIES_2_001,CA_1,HOBBIES_2_001_CA_1_validation,d_3,0,11101,5.468750,2011-01-31,Monday,3,1,2011,5.468750,0.0,5
3,HOBBIES_2,HOBBIES_2_001,CA_1,HOBBIES_2_001_CA_1_validation,d_4,0,11101,5.468750,2011-02-01,Tuesday,4,2,2011,5.468750,0.0,5
4,HOBBIES_2,HOBBIES_2_001,CA_1,HOBBIES_2_001_CA_1_validation,d_5,0,11101,5.468750,2011-02-02,Wednesday,5,2,2011,5.468750,0.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2174081,HOBBIES_2,HOBBIES_2_149,WI_3,HOBBIES_2_149_WI_3_validation,d_1909,2,11612,0.970215,2016-04-20,Wednesday,5,4,2016,2.470703,1.5,16
2174082,HOBBIES_2,HOBBIES_2_149,WI_3,HOBBIES_2_149_WI_3_validation,d_1910,0,11612,0.970215,2016-04-21,Thursday,6,4,2016,2.470703,1.5,16
2174083,HOBBIES_2,HOBBIES_2_149,WI_3,HOBBIES_2_149_WI_3_validation,d_1911,0,11612,0.970215,2016-04-22,Friday,7,4,2016,2.470703,1.5,16
2174084,HOBBIES_2,HOBBIES_2_149,WI_3,HOBBIES_2_149_WI_3_validation,d_1912,0,11613,0.970215,2016-04-23,Saturday,1,4,2016,2.470703,1.5,16


In [ ]:
food_master.drop(food_master.columns[0], axis=1, inplace=True)

In [ ]:
food_master = pd.merge(food_master, calendar, on=['d'],how='left')

In [ ]:
food_master

,item_id,store_id,id,d,Quantity_sold,wm_yr_wk_x,sell_price,date_x,weekday,wday,month,year,mrp,discount,week_number,date_y,wm_yr_wk_y,event_name_1,PresidentsDay,LentStart,LentWeek2,StPatricksDay,Purim End,OrthodoxEaster,Pesach End,Cinco De Mayo,Mother's day,MemorialDay,NBAFinalsStart,NBAFinalsEnd,Father's day,IndependenceDay,Ramadan starts,Eid al-Fitr,LaborDay,ColumbusDay,Halloween,EidAlAdha,VeteransDay,Thanksgiving,Christmas,Chanukah End,NewYear,OrthodoxChristmas,MartinLutherKingDay,Easter
0,HOBBIES_2_001,CA_1,HOBBIES_2_001_CA_1_validation,d_1,1,11101,5.468750,2011-01-29,Saturday,1,1,2011,5.468750,0.0,4,2011-01-29,11101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,HOBBIES_2_001,CA_1,HOBBIES_2_001_CA_1_validation,d_2,0,11101,5.468750,2011-01-30,Sunday,2,1,2011,5.468750,0.0,4,2011-01-30,11101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,HOBBIES_2_001,CA_1,HOBBIES_2_001_CA_1_validation,d_3,0,11101,5.468750,2011-01-31,Monday,3,1,2011,5.468750,0.0,5,2011-01-31,11101,NaN,-21.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,HOBBIES_2_001,CA_1,HOBBIES_2_001_CA_1_validation,d_4,0,11101,5.468750,2011-02-01,Tuesday,4,2,2011,5.468750,0.0,5,2011-02-01,11101,NaN,-20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,HOBBIES_2_001,CA_1,HOBBIES_2_001_CA_1_validation,d_5,0,11101,5.468750,2011-02-02,Wednesday,5,2,2011,5.468750,0.0,5,2011-02-02,11101,NaN,-19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2174081,HOBBIES_2_149,WI_3,HOBBIES_2_149_WI_3_validation,d_1909,2,11612,0.970215,2016-04-20,Wednesday,5,4,2016,2.470703,1.5,16,2016-04-20,11612,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2174082,HOBBIES_2_149,WI_3,HOBBIES_2_149_WI_3_validation,d_1910,0,11612,0.970215,2016-04-21,Thursday,6,4,2016,2.470703,1.5,16,2016-04-21,11612,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2174083,HOBBIES_2_149,WI_3,HOBBIES_2_149_WI_3_validation,d_1911,0,11612,0.970215,2016-04-22,Friday,7,4,2016,2.470703,1.5,16,2016-04-22,11612,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2174084,HOBBIES_2_149,WI_3,HOBBIES_2_149_WI_3_validation,d_1912,0,11613,0.970215,2016-04-23,Saturday,1,4,2016,2.470703,1.5,16,2016-04-23,11613,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
food_master.drop(['date_x', 'date_y', 'wm_yr_wk_y'], axis=1, inplace=True)

In [ ]:
food_master.to_parquet(path = (input_dir+'food3.gzip'), compression='gzip', partition_cols=['dept_id', 'item_id', 'store_id'])

ArrowNotImplementedError: ignored

In [ ]:
food_master.to_csv(input_dir + 'hobbies2_master.csv')

In [ ]:
food_master.drop(food_master.columns[0], axis=1, inplace=True)

In [ ]:
def set_td_weights(df, date_col):
    df[date_col] = pd.to_datetime(df[date_col])
    df['td'] = pd.to_timedelta(df[date_col]).dt.days
    df['td'] = df['td'] - df['td'].min()
    decay_const = 365. # number of days after the weight is down to 0.5
    const = -np.log(0.5) / decay_const
    df['weights_td'] = np.exp((df['td'] - df['td'].max()) * const)
    return df